In [1]:
import os

In [2]:
%pwd

'/Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tejred213"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bf81a71e3c7bbf104640edcc945d285b17ae08e3"

In [6]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
# %pip install mlflow

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.10
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        # Fixed `model` to `self.model`
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Handle model logging based on registry type
            try:
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(
                        self.model, "model", registered_model_name="VGG16Model"
                    )
                else:
                    mlflow.keras.log_model(self.model, "model")
            except ModuleNotFoundError as e:
                # Replace distutils usage with setuptools if possible
                if "distutils" in str(e):
                    print("Switching to setuptools due to distutils issue.")
                    import setuptools  # Ensure setuptools is installed
                else:
                    raise e


# Main execution
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except ModuleNotFoundError as e:
    if "distutils" in str(e):
        print("Distutils is missing. Please install setuptools and try again.")
        import subprocess
        subprocess.check_call(["pip", "install", "setuptools"])
    else:
        raise e

except Exception as e:
    raise e


[2025-01-27 22:33:33,801:INFO:common:yaml file: config/config.yaml loaded successfully]
[2025-01-27 22:33:33,802:INFO:common:yaml file: params.yaml loaded successfully]
[2025-01-27 22:33:33,803:INFO:common:created directory at: artifacts]
An error occurred: No file or directory found at artifacts/training/model.h5


In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tejred213"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bf81a71e3c7bbf104640edcc945d285b17ae08e3"

import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    
# %pip install mlflow

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.10
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        # Fixed `model` to `self.model`
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Handle model logging based on registry type
            try:
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(
                        self.model, "model", registered_model_name="VGG16Model"
                    )
                else:
                    mlflow.keras.log_model(self.model, "model")
            except ModuleNotFoundError as e:
                # Replace distutils usage with setuptools if possible
                if "distutils" in str(e):
                    print("Switching to setuptools due to distutils issue.")
                    import setuptools  # Ensure setuptools is installed
                else:
                    raise e


# Main execution
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except ModuleNotFoundError as e:
    if "distutils" in str(e):
        print("Distutils is missing. Please install setuptools and try again.")
        import subprocess
        subprocess.check_call(["pip", "install", "setuptools"])
    else:
        raise e

except Exception as e:
    raise e


[2025-01-27 22:25:38,541:WARNING:optimizer:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.]
[2025-01-27 22:25:38,553:INFO:common:yaml file: config/config.yaml loaded successfully]
[2025-01-27 22:25:38,554:INFO:common:yaml file: params.yaml loaded successfully]
[2025-01-27 22:25:38,554:INFO:common:created directory at: artifacts]
[2025-01-27 22:25:38,975:WARNING:optimizer:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 3s 312ms/step - loss: 0.6021 - accuracy: 0.7941
[2025-01-27 22:25:41,655:INFO:common:json file saved at: scores.json]


2025/01/27 22:25:42 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/br/8xndxmrd1p74dp71tt814grw0000gn/T/tmpaldxu_l6/model/data/model/assets
[2025-01-27 22:25:46,563:INFO:builder_impl:Assets written to: /var/folders/br/8xndxmrd1p74dp71tt814grw0000gn/T/tmpaldxu_l6/model/data/model/assets]


2025/01/27 22:25:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/01/27 22:26:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 7
Created version '7' of model 'VGG16Model'.


🏃 View run handsome-toad-365 at: https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow/#/experiments/0/runs/08f38de1b12f4c7793d3272c31faf04a
🧪 View experiment at: https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow/#/experiments/0


In [15]:
# class Evaluation:
#     def __init__(self, config: EvaluationConfig):
#         self.config = config

#     def _valid_generator(self):
#         datagenerator_kwargs = dict(
#             rescale=1.0 / 255,
#             validation_split=0.30
#         )

#         dataflow_kwargs = dict(
#             target_size=self.config.params_image_size[:-1],
#             batch_size=self.config.params_batch_size,
#             interpolation="bilinear"
#         )

#         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#             **datagenerator_kwargs
#         )

#         self.valid_generator = valid_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="validation",
#             shuffle=False,
#             **dataflow_kwargs
#         )

#     @staticmethod
#     def load_model(path: Path) -> tf.keras.Model:
#         return tf.keras.models.load_model(path)

#     def evaluation(self):
#         self.model = self.load_model(self.config.path_of_model)
#         self._valid_generator()
#         # Fixed `model` to `self.model`
#         self.score = self.model.evaluate(self.valid_generator)
#         self.save_score()

#     def save_score(self):
#         scores = {"loss": self.score[0], "accuracy": self.score[1]}
#         save_json(path=Path("scores.json"), data=scores)

#     def log_into_mlflow(self):
#         mlflow.set_registry_uri(self.config.mlflow_uri)
#         tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

#         with mlflow.start_run():
#             mlflow.log_params(self.config.all_params)
#             mlflow.log_metrics(
#                 {"loss": self.score[0], "accuracy": self.score[1]}
#             )
#             # Model registry does not work with file store
#             if tracking_url_type_store != "file":
#                 # Register the model
#                 mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
#             else:
#                 mlflow.keras.log_model(self.model, "model")


In [ ]:
# try:
#     config = ConfigurationManager()
#     eval_config = config.get_evaluation_config()
#     evaluation = Evaluation(eval_config)
#     evaluation.evaluation()
#     evaluation.log_into_mlflow()

# except Exception as e:
#    raise e

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tejred213"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bf81a71e3c7bbf104640edcc945d285b17ae08e3"

import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    
# %pip install mlflow

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.10
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            class_mode='categorical',  # Explicitly set class_mode  # Explicitly define class order
            **dataflow_kwargs
        )
        
        # Print class indices to verify the order
        print("Class indices:", self.valid_generator.class_indices)

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        # Fixed `model` to `self.model`
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Handle model logging based on registry type
            try:
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(
                        self.model, "model", registered_model_name="VGG16Model"
                    )
                else:
                    mlflow.keras.log_model(self.model, "model")
            except ModuleNotFoundError as e:
                # Replace distutils usage with setuptools if possible
                if "distutils" in str(e):
                    print("Switching to setuptools due to distutils issue.")
                    import setuptools  # Ensure setuptools is installed
                else:
                    raise e


# Main execution
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except ModuleNotFoundError as e:
    if "distutils" in str(e):
        print("Distutils is missing. Please install setuptools and try again.")
        import subprocess
        subprocess.check_call(["pip", "install", "setuptools"])
    else:
        raise e

except Exception as e:
    raise e
